In [ ]:
from keras.applications import inception_v3
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import numpy as np
import pandas as pd
from keras.preprocessing.image import load_img, save_img, img_to_array
import glob as glob
import scipy
import argparse
import sys, os

from keras.applications import inception_v3, resnet50
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions, preprocess_input
K.set_learning_phase(0)


In [ ]:
# Load the model and include the "top" classification layer
#model = inception_v3.InceptionV3(weights='imagenet',include_top=True)
model = inception_v3.InceptionV3(weights='imagenet')

In [ ]:
def preprocess_image(image_path):
    # Util function to open, resize and format pictures
    # into appropriate tensors.
    img = load_img(image_path)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img


def deprocess_image(x):
    # Util function to convert a tensor into a valid image.
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x


In [ ]:
# Should output /device:GPU:0
K.tensorflow_backend._get_available_gpus()

In [ ]:
count = 0
pred_scuss = 0
pred_error = 0

# The path of test image 
in_dir = 'imagenet_attacked/defense/'
in_list = glob.glob(in_dir + '*.png')
for in_name in in_list:    
    true_class = int(os.path.basename(in_name).split('_')[1])
    in_image = preprocess_image(in_name)    
    probs = model.predict(in_image)
    pred_class = np.argmax(probs)
    if(true_class == pred_class):        
        pred_scuss += 1
        print(in_name,true_class, pred_class, probs[0][pred_class])
    else:
        pred_error += 1
        print(in_name,true_class, pred_class, probs[0][pred_class])
        d_img = deprocess_image(in_image)
        out_name = in_name + '_failed.png'
        save_img(out_name, d_img)
        

print(pred_scuss, pred_error, str(pred_scuss/(len(in_list))*100)+'%')